In [1]:
include("../src/Utero.jl")
using .Utero 

In [2]:
ctx = ⬅Context()
ForwardBackward!(ctx, x -> 2.0*x + x, AddParams!(ctx, 7.0))

(21.0, Any[3.0])

In [3]:
# should be 12.8
ctx = ⬅Context()
ForwardBackward!(ctx, x -> x^2.0 + x^2.0, AddParams!(ctx, 3.2)) 

(20.480000000000004, Any[12.8])

In [4]:
# Gradient should be 56.21
ctx = Utero.⬅Context()
ForwardBackward!(ctx, w -> 2.0 * (((2.0*w + 12.0) - w) * w) + sin(w^2.0), AddParams!(ctx, 7.0))

(265.0462473472405, Any[56.20829561241092])

In [5]:
# should be approx. (27.53, 3.0428)
ctx = Utero.⬅Context()
x, y = AddParams!(ctx, 13.2, 89.0)
ForwardBackward!(ctx, (x, y) -> (cos(y)^3 * 4sin(x))^2 + cos(x*y)^2, x, y) 

(1.0748674472530375, Any[27.53002464314667, 3.0428042273213602])

In [6]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W[5:5]' * W[5:5], 3)

(0.009786959912324152, Any[sparse([5], [1], [0.19785813010664133], 30, 1)])

In [7]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W' * W, 3)
print(size(grads[1]))

(30, 30)

In [8]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W' * W, 3)
print(size(grads[1]))

(30, 30)

In [9]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(2,1))
(out, grads) = ForwardBackward!(ctx, _ -> W .* 2, nothing)
@show size(W.val)
print(size(grads[1]))
print(grads[1])


size(W.val) = (2, 1)
(2, 1)

[2.0; 2.0;;]

In [10]:
W = rand(2,1)
@show ((W' * W) .* W)' * W

ctx = Utero.⬅Context()
W1, W2, W3, W4 = AddParams!(ctx, rand(2,1), rand(2,1), rand(2,1), rand(2,1))
(out, grads) = ForwardBackward!(ctx, _ -> ((W1' * W2) .* W3)' * W4, nothing)
@show size(W.val)
print(size(grads[1]))
print(grads[1])

((W' * W) .* W)' * W = [0.013959140269445931;;]
used

DimensionMismatch: DimensionMismatch: A has dimensions (2,2) but B has dimensions (1,2)

In [11]:
ctx = Utero.⬅Context()
W1, W2, W3 = AddParams!(ctx, rand(30,30), rand(40,30), rand(30,40))

function model(X)
    return  ReLU(W3 * ReLU(W2 * ReLU(W1 * X)))
end

X = rand(1500, 30)
Y = mapslices(row -> cos.(row), X, dims=2) 

println(ctx.Params[1].val)
for (x, y) in zip(eachrow(X), eachrow(Y))
    println(MeanSquaredError(model(x), y).val)
    (out, grads) = ForwardBackward!(ctx, x -> MeanSquaredError(model(x), y), x)
    GradientStep!(0.0001, ctx.Params, grads)
    exit("")
end
println(ctx.Params[1].val)


[0.8807340106611806 0.7028628181512904 0.6993603874307783 0.046542965169035555 0.747464327555298 0.20418330172245436 0.28528790226085854 0.9726261445740348 0.6019728751227142 0.3100016516099162 0.09769014367550954 0.7844435624812808 0.39216724227898103 0.6574388927880871 0.405148123371853 0.9798239215460379 0.030121040733758764 0.6872171275024377 0.8154626762689816 0.1222054673952564 0.8963922219982785 0.5888980722156545 0.5933718546132705 0.6805658182186544 0.3722216666942615 0.8039306567019393 0.1031769865152713 0.25702505850025303 0.2051666611693449 0.43383740208660015; 0.6598388342631529 0.7837189387999985 0.13663767676728178 0.7427971857916956 0.917067867775016 0.8821239238633304 0.7471127285258506 0.6320842164413141 0.8148062267206214 0.31872718344588447 0.016853142125011877 0.32861824690469554 0.4686933107817026 0.4269500447062773 0.39412123256634723 0.6055741092989865 0.7201085793303992 0.46985458882172226 0.3604100864308635 0.7733284258621406 0.38094091719994294 0.790879053309

 0.3037775413919678 0.5312679071741274 0.8821810114352534 0.7963787526301627 0.9720261842697284 0.7567341627964236 0.11556032691499851 0.20166651250807255 0.2529646653395009 0.11140976197227281; 0.58002517102756 0.5852373772700988 0.5852038379936227 0.08453041434559683 0.5655022473830725 0.293894656927162 0.6318494012019631 0.8849008096345662 0.9967709452310516 0.7366948596042842 0.08331567737083245 0.9015227850936413 0.05284220795795558 0.3820152843770328 0.1505566020484831 0.7226135370834892 0.06534880098453089 0.9673436185120959 0.8579999171295963 0.614430856386071 0.43213622115878947 0.20382330320721798 0.8471703010881897 0.9423012928154338 0.07583793183407894 0.7737598866103552 0.48450034518190166 0.8088482634828996 0.5456141475437741 0.013324213687944342; 0.24914526338042586 0.690585622608494 0.5371396446553749 0.5322107493362458 0.5160327447374277 0.6466084157592207 0.5190077389572147 0.9674002718068763 0.6040567633409849 0.6999895674847811 0.32098632965412743 0.0899198561887488

1.6604412116902635e8


DimensionMismatch: DimensionMismatch: A has dimensions (30,30) but B has dimensions (1,40)

In [12]:
using MLDatasets

TrainData = MNIST(:train)
ValidData = MNIST(:test)

ctx = ⬅Context()

densenet = Sequential(Dense(28*28, 16, ReLU), Dense(16, 16, ReLU), Dense(16, 10, ReLU), Softmax)
#display(densenet(rand(28*28, 1)))
#display(MeanSquaredError(densenet(rand(28*28, 1)), rand(10, 1)))
AddParams!(ctx, densenet)
println(typeof(densenet.Sequence[1].W))
for (x, y) in DataIterator(TrainData.features, TrainData.targets)
    x = reshape(x, 28*28, 1)
    y = OneHot(Int64(y+1), 10)
    y = reshape(y, 10, 1)
    display(densenet.Sequence[1].W)
    (out, paramgrads) = ForwardBackward!(ctx, x -> densenet(x), x)
    GradientStep!(0.0001, ctx.Params, paramgrads)
    display(paramgrads[1])
    display(densenet.Sequence[1].W)
    exit("")
end

Main.Utero.⬅Tracker{Matrix{Float64}}([-0.17454839331222996 -1.886314971077164 … 1.375765785900401 -3.439794321264979; -0.7397017191593996 -2.0015716851520344 … -0.4882506351356773 -0.5032782763923097; … ; 0.1344650414725121 -1.2583910614301344 … 1.0596248787121916 -0.7061083072721062; 0.22922415411027136 -0.5249960187669902 … 0.18293253532817827 -0.024233552185484238], UUID("2a6732b2-c204-11ee-06a8-67959324a222"), Any[], Any[])

Main.Utero.⬅Tracker{Matrix{Float64}}


used

DimensionMismatch: DimensionMismatch: A has dimensions (10,1) but B has dimensions (10,1)